
Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [26]:
KEYWORDS = ['python', 'парсинг']

In [50]:
import requests
import time
from bs4 import BeautifulSoup

def get_all_links(url, query, pages):
    urls = []
    params = {
        'q': query
    }
    for i in range(pages):
        a = f'page{i}/'
        url1 = url + a
        #print(url1)
        req = requests.get(url1, params)
        soup = BeautifulSoup(req.text, 'html.parser')
        posts = soup.find_all('h2', class_='post__title')
        #print(posts)
        urls += list(map(lambda x: x.find('a').get('href'), posts))
        time.sleep(0.2)
    return urls

all_links = get_all_links('http://habr.com/ru/all/', KEYWORDS, 16)

In [52]:
import pandas as pd

habr_news = pd.DataFrame()

for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.2)
    habr_news['date'] = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
    habr_news['title'] = soup.find('span', class_='post__title-text').text
    habr_news['text'] = soup.find('div', class_='post__body post__body_full').text

habr_news.head()

,date,title,text


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [69]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [70]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [74]:
import json
result = []
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast','Vaar-Version': '0'}
addresses = {'emailAddresses': EMAIL}
res2 = requests.post(URL, json=addresses, headers=headers)
res2.json()
result['date'] = pd.DataFrame(res2.json()['publishDate'])
result['site'] = pd.DataFrame(res2.json()['site'])
result['description'] = pd.DataFrame(res2.json()['description'])
result

TypeError: 'NoneType' object is not subscriptable